In [ ]:
import numpy as np
from itertools import product
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# Define the NeuralNetwork class
class NeuralNetwork:
    def __init__(self,data, input_size, hidden_size, output_size, activation='sigmoid', learning_rate=0.1):
        # Initialize the neural network architecture
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.data = data

        # Initialize weights and biases
        self.weights_input_hidden = np.random.rand(input_size, hidden_size)
        self.bias_hidden = np.zeros((1, hidden_size))
        self.weights_hidden_output = np.random.rand(hidden_size, output_size)
        self.bias_output = np.zeros((1, output_size))

        # Set the activation function
        self.activation = self.sigmoid if activation == 'sigmoid' else \
                          self.tanh if activation == 'tanh' else \
                          self.relu
        self.activation_derivative = self.sigmoid_derivative if activation == 'sigmoid' else \
                          self.tanh_derivative if activation == 'tanh' else \
                          self.relu_derivative

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def tanh(self, x):
        return np.tanh(x)

    def tanh_derivative(self, x):
        return 1 - x**2

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return (x > 0).astype(float)


    #As the first step, pre-process and clean your dataset. There should be a
    #method that does this.
    # Data Preprocessing
    def preprocess_and_clean_train(self,data):
      #In data we have 1 duplicate record so we are dropping the Records
      self.data.drop_duplicates(inplace=True)
      #No null values in the data
      # data.isnull().sum()
      X = self.data.drop(columns=['output'])
      y = self.data['output']
      # Standardize the features
      scaler = StandardScaler()
      X = scaler.fit_transform(X)
      return X, np.array(y)
    def preprocess_and_clean_test(self,X):
      # Standardize the features
      scaler = StandardScaler()
      X = scaler.fit_transform(X)
      return X


    def split_data(self, X, y, test_size, random_state=None):
        # Split the dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        return X_train, X_test, y_train, y_test

    # #Code a neural net having at least one hidden layer. You are free to
    # select the number of neurons in each layer. Each neuron in the hidden
    # and output layers should have a bias connection.
    def forward(self, X):
        # Perform forward propagation with the selected activation function
        self.hidden_layer_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_layer_output = self.activation(self.hidden_layer_input)
        self.output_layer_input = np.dot(self.hidden_layer_output, self.weights_hidden_output) + self.bias_output
        self.output_layer_output = self.activation(self.output_layer_input)
        return self.output_layer_output

    def backward(self, X, y, output):
        # Calculate the loss
        error = y - output

        # Compute gradients based on the activation function's derivative
        delta_output = error * self.activation_derivative(output)
        error_hidden = delta_output.dot(self.weights_hidden_output.T)
        delta_hidden = error_hidden * self.activation_derivative(self.hidden_layer_output)

        # Update weights and biases using SGD
        self.weights_hidden_output += self.hidden_layer_output.T.dot(delta_output) * self.learning_rate
        self.bias_output += np.sum(delta_output, axis=0, keepdims=True) * self.learning_rate
        self.weights_input_hidden += X.T.dot(delta_hidden) * self.learning_rate
        self.bias_hidden += np.sum(delta_hidden, axis=0, keepdims=True) * self.learning_rate

    def train(self, data,epochs):
        X, y = self.preprocess_and_clean_train(data)  # Pre-process and clean the input data
        X_train, _, y_train, _ = self.split_data(X, y, test_size=0.2, random_state=42)  # Split the data for training
        for _ in range(epochs):
            for i in range(len(X_train)):
                input_data = X_train[i:i+1]
                target = y_train[i:i+1]
                output = self.forward(input_data)
                self.backward(input_data, target, output)

    def test(self, X):
        X = self.preprocess_and_clean_test(X)  # Pre-process the input data
        return np.round(self.forward(X))

    def accuracy(self, X, y):
        predictions = self.test(X)
        return np.mean(predictions == y)




data = pd.read_csv('https://raw.githubusercontent.com/somanathvamshi/Minist_rnn/main/heart.csv')

# Create a DataFrame to store the results
results = pd.DataFrame(columns=['Parameters', 'Training Accuracy', 'Test Accuracy'])

# Create a list of parameter combinations to try
param_grid = {
    'activation': ['sigmoid', 'tanh', 'relu'],
    'learning_rate': [0.01, 0.1, 0.2],
    'hidden_size': [4, 8, 16]
}

# Iterate over all parameter combinations
for params in product(*param_grid.values()):
    activation, learning_rate, hidden_size = params
    nn = NeuralNetwork(data = data,input_size=13, hidden_size=hidden_size, output_size=1, activation=activation, learning_rate=learning_rate)
    X, y = nn.preprocess_and_clean_train(data)
    #Split the pre-processed dataset into training and testing parts
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the neural network using SGD
    nn.train(data, epochs=10000)

    # Calculate training accuracy
    training_accuracy = nn.accuracy(X_train, y_train)

    # Calculate test accuracy
    test_accuracy = nn.accuracy(X_test, y_test)

    # Store the results in the DataFrame
    results = results.append({'Parameters': params, 'Training Accuracy': training_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)

# Display the results in a tabular format
results_df = pd.DataFrame.from_dict(results)
results_df

aa
bb
cc
cc
aa
bb


<ipython-input-52-55159efd9f18>:160: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Parameters': params, 'Training Accuracy': training_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)


cc
cc
aa
bb


<ipython-input-52-55159efd9f18>:160: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Parameters': params, 'Training Accuracy': training_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)


cc
cc
aa
bb


...

cc
cc
aa
bb


<ipython-input-52-55159efd9f18>:160: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Parameters': params, 'Training Accuracy': training_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)


cc
cc


<ipython-input-52-55159efd9f18>:160: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Parameters': params, 'Training Accuracy': training_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)


,Parameters,Training Accuracy,Test Accuracy
0,"(sigmoid, 0.01, 4)",0.503366,0.497984
1,"(sigmoid, 0.01, 8)",0.504554,0.501209
2,"(sigmoid, 0.01, 16)",0.504158,0.500403
3,"(sigmoid, 0.1, 4)",0.502970,0.501209
4,"(sigmoid, 0.1, 8)",0.504158,0.497984
5,"(sigmoid, 0.1, 16)",0.504950,0.503628
6,"(sigmoid, 0.2, 4)",0.501782,0.497984
7,"(sigmoid, 0.2, 8)",0.504950,0.500403
8,"(sigmoid, 0.2, 16)",0.504158,0.500403
9,"(tanh, 0.01, 4)",0.505656,0.477829


In [ ]:
results_df.to_csv('results.csv')